# 인턴 교육 2주차 과제: Selenium을 이용한 유튜브 댓글 크롤링
- 과제 기한: 6월 21일까지
- 제출자: 배정환 인턴

**세부과제**

1. n_scroll 변수를 활용하여 스크롤 다운 5번
    - 스크롤 다운 함수를 만든다.

2. Pandas를 이용한 데이터프레임 저장
    - `columns: [UserID, 댓글텍스트, 좋아요 수]`
    - 모든 for문이 끝난 후 URL에서 video_id 추출해서 videoID column 추가
    - csv로 저장

3. 전체 함수 만들기
    - URL만 입력하면 바로 저장될 수 있도록 함수를 만든다.

**목표**
- 스크롤하면서 로딩되는 댓글들을 Selenium 크롤링을 이용해 수집하기
- 에러에 대한 처리를 하며 나중에 오랜만에 했을 때 생기는 오류에 대해서도 빠르고 쉽게 처리할 수 있도록 경험을 쌓는다.

**팁**
- [Selenium Documentation](https://www.selenium.dev/documentation/)
- [Selenium-Python Documentation](https://selenium-python.readthedocs.io/index.html)